In [120]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

#######딥러닝 라이브러리##########
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN, Dropout

tf.keras.backend.set_floatx('float64')

In [50]:
train=pd.read_csv('./Data_for_Modeling/Train_for_model.csv')
#train['date_time'] = pd.to_datetime(train['date_time'])

test=pd.read_csv('./Data_for_Modeling/Test_for_model.csv')
#test['date_time'] = pd.to_datetime(test['date_time'])

submission=pd.read_csv('sample_submission.csv', encoding = 'cp949')

In [51]:
#train.shape 122400 X 10
#60개의 건물 X 85일 24시간 =122400
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
118075,60,2020-08-21 19,3780.000,27.0,1.9,77.0,0.0,0.1,1,1
118076,60,2020-08-21 20,3691.440,26.0,2.6,82.0,0.0,0.0,1,1
118077,60,2020-08-21 21,3532.896,25.2,2.2,83.0,0.0,0.0,1,1
118078,60,2020-08-21 22,3411.504,24.9,2.4,84.0,0.0,0.0,1,1


In [52]:
#test.shape 10080 X 9
#60개의 건물 X 7일 24시간 =10080
test

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-08-22 00,8646.264,25.2,1.9,87,0.0,0.0,0,0
1,1,2020-08-22 01,8637.192,25.0,2.0,86,0.0,0.0,0,0
2,1,2020-08-22 02,8616.456,25.0,1.4,86,0.0,0.0,0,0
3,1,2020-08-22 03,8592.480,24.7,1.8,86,0.0,0.0,0,0
4,1,2020-08-22 04,8570.448,24.4,1.7,87,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
4315,60,2020-08-24 19,4114.368,27.8,2.3,68,0.0,0.7,1,1
4316,60,2020-08-24 20,3975.696,27.3,1.2,71,0.0,0.0,1,1
4317,60,2020-08-24 21,3572.208,27.3,1.8,71,0.0,0.0,1,1
4318,60,2020-08-24 22,3299.184,27.1,1.8,74,0.0,0.0,1,1


# **딥러닝 모델(전력사용량만 변수로 사용)**

In [53]:
import datetime

In [54]:
train['THI'] = 9/5*train['기온(°C)'] - 0.55*(1-train['습도(%)']/100)*(9/5*train['기온(°C)']-26)+32
test['THI'] = 9/5*test['기온(°C)'] - 0.55*(1-test['습도(%)']/100)*(9/5*test['기온(°C)']-26)+32

In [55]:
# renaming columns
train.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag','thi']
test.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag','thi']

In [56]:
eda_df = train.copy()

In [57]:
# adding features related to datetime
eda_df['datetime'] = pd.to_datetime(eda_df['datetime'])
eda_df['hour'] = eda_df['datetime'].dt.hour
eda_df['weekday'] = eda_df['datetime'].dt.weekday
eda_df['date'] = eda_df['datetime'].dt.date
eda_df['day'] = eda_df['datetime'].dt.day
eda_df['month'] = eda_df['datetime'].dt.month
eda_df['weekend'] = eda_df['weekday'].isin([5,6]).astype(int)

In [58]:
from sklearn.cluster import k_means
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 건물을 기준으로 하는 data frame 생성
by_weekday = eda_df.groupby(['num','weekday'])['target'].median().reset_index().pivot('num','weekday','target').reset_index()
by_hour = eda_df.groupby(['num','hour'])['target'].median().reset_index().pivot('num','hour','target').reset_index().drop('num', axis = 1)
df = pd.concat([by_weekday, by_hour], axis= 1)
columns = ['num'] + ['day'+str(i) for i in range(7)] + ['hour'+str(i) for i in range(24)]
df.columns = columns
for i in range(len(df)):
    # 요일 별 전력 중앙값에 대해 scaling
    df.iloc[i,1:8] = (df.iloc[i,1:8] - df.iloc[i,1:8].mean())/df.iloc[i,1:8].std()
    # 시간대별 전력 중앙값에 대해 scaling
    df.iloc[i,8:] = (df.iloc[i,8:] - df.iloc[i,8:].mean())/df.iloc[i,8:].std()

In [59]:
##############전력사용량(kWh) 정규화####################################

mini=train.iloc[:,2].min()
size=train.iloc[:,2].max()-train.iloc[:,2].min()
train.iloc[:,2]=(train.iloc[:,2]-mini)/size


col=train.columns
scaler = StandardScaler()

train.iloc[:,3:]= scaler.fit_transform(train.iloc[:,3:])
train = pd.DataFrame(data = train,columns =col)

test.iloc[:,3:]= scaler.transform(test.iloc[:,3:])
test = pd.DataFrame(data = test,columns =col)


In [60]:
train

,num,datetime,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool_flag,solar_flag,thi
0,1,2020-06-01 00,0.461072,-1.931816,0.218064,0.755834,0.104279,-0.570798,-1.468977,-0.967204,-2.072800
1,1,2020-06-01 01,0.458624,-1.902416,0.480203,0.691868,-0.084610,-0.570798,-1.468977,-0.967204,-2.043530
2,1,2020-06-01 02,0.457017,-1.961215,0.676807,0.691868,-0.197944,-0.570798,-1.468977,-0.967204,-2.115145
3,1,2020-06-01 03,0.453729,-2.078813,0.676807,0.691868,-0.197944,-0.570798,-1.468977,-0.967204,-2.258374
4,1,2020-06-01 04,0.453437,-2.108212,0.742342,0.755834,-0.197944,-0.570798,-1.468977,-0.967204,-2.288887
...,...,...,...,...,...,...,...,...,...,...,...
118075,60,2020-08-21 19,0.213087,0.831729,-0.175145,-0.203656,-0.197944,-0.299864,0.680746,1.033908,0.922344
118076,60,2020-08-21 20,0.208095,0.537735,0.283598,0.116174,-0.197944,-0.570798,0.680746,1.033908,0.712992
118077,60,2020-08-21 21,0.199157,0.302540,0.021459,0.180140,-0.197944,-0.570798,0.680746,1.033908,0.463737
118078,60,2020-08-21 22,0.192314,0.214341,0.152529,0.244106,-0.197944,-0.570798,0.680746,1.033908,0.382951


In [61]:
train=train.drop(train.iloc[:,:2], axis=1)
test=test.drop(test.iloc[:,:2], axis=1)
train=train.drop(train.iloc[:,1:8], axis=1)
test=test.drop(test.iloc[:,1:8], axis=1)

In [62]:
train

,target,thi
0,0.461072,-2.072800
1,0.458624,-2.043530
2,0.457017,-2.115145
3,0.453729,-2.258374
4,0.453437,-2.288887
...,...,...
118075,0.213087,0.922344
118076,0.208095,0.712992
118077,0.199157,0.463737
118078,0.192314,0.382951


In [138]:
train['target']=train['thi']*0.1+train['target']*0.9
test['target']=test['thi']*0.1+train['target']*0.9

In [140]:
train=train.drop(train.iloc[:,1:], axis=1)
test=test.drop(test.iloc[:,1:], axis=1)

In [141]:
train

,target
0,0.207685
1,0.208409
2,0.199801
3,0.182519
4,0.179205
...,...
118075,0.284013
118076,0.258584
118077,0.225615
118078,0.211378


In [142]:
####################################################

input_window = 996 #임의의 수
output_window = 24 #168 7일 24시간
window = 12 #window는 12시간 마다는 12시간 마다
num_features = 1 #베이스라인은 feature를 하나만 사용했습니다.
num_power = 60 #건물개수
end_=168 
lstm_units=32
dropout=0.5
EPOCH=60
BATCH_SIZE=128

In [143]:
#train을 tensor로 변경 (60, 24*82, 1)
train_x=tf.reshape(train.values, [num_power, -1, num_features])
print(f'train_x.shape:{train_x.shape}')

train_x.shape:(60, 1968, 1)


In [144]:
#train_window_x np.zeros를 만듬 (60, 79, 996, 8)
train_window_x= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, input_window, num_features)) 
train_window_y= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, output_window, num_features))
print(f'train_window_x.shape:{train_window_x.shape}')
print(f'train_window_y.shape:{train_window_y.shape}')

train_window_x.shape:(60, 79, 996, 1)
train_window_y.shape:(60, 79, 24, 1)


In [145]:
#train_window_x에 train값 채워넣기
for example in range(train_x.shape[0]):
    
    for start in range(0, train_x.shape[1]-(input_window+output_window), window):
        end=start+input_window
        train_window_x[example, start//window, :] = train_x[example, start: end               , :]
        train_window_y[example, start//window, :] = train_x[example, end  : end+ output_window, :]

In [146]:
#new_train_x, reshape통해 lstm에 알맞은 형태로 집어넣기
new_train_x=tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y=tf.reshape(train_window_y, [-1, output_window,num_features])
print(f'new_train_x.shape:{new_train_x.shape}')
print(f'new_train_y.shape:{new_train_y.shape}')


new_train_x.shape:(4740, 996, 1)
new_train_y.shape:(4740, 24, 1)


In [147]:
from sklearn.model_selection import KFold
from numpy.random import seed

In [148]:
# Custom Loss Function

import tensorflow.keras.backend as K

def SMAPE(true,predicted):
    epsilon = 0.1
    summ = K.maximum(K.abs(true) + K.abs(predicted) + epsilon, 0.5 + epsilon)
    smape = K.abs(predicted - true) / summ * 2.0
    return smape

In [149]:
[output_window, num_features]

[24, 1]

In [157]:
#####층 쌓기###########


# model=Sequential()
# LSTM(lstm_units, return_sequences=False, recurrent_dropout=dropout),
# LSTM(64, return_sequences=False, recurrent_dropout=dropout),
# Dense(output_window * num_features, kernel_initializer=tf.initializers.zeros()), 
# Dense(24, kernel_initializer=tf.initializers.zeros()), 
# Dropout(0.5),
# Reshape([output_window, num_features])
model=Sequential([
LSTM(lstm_units, return_sequences=False, recurrent_dropout=dropout),
Dense(output_window * num_features, kernel_initializer=tf.initializers.zeros()),


Reshape([output_window, num_features])
])
model.compile(optimizer='adam', loss='mape')


#######Compile 구성하기################


# # 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
# class PrintDot(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs):
#         if epoch % 10 == 0: print('')
#         print('.', end='')

#가장 좋은 성능을 낸 val_loss가 적은 model만 남겨 놓았습니다.
#save_best_only=tf.keras.callbacks.ModelCheckpoint(filepath="lstm_model.h5", monitor='val_loss', save_best_only=True)


early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

#검증 손실이 10epoch 동안 좋아지지 않으면 학습률을 0.1 배로 재구성하는 명령어입니다.
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4)


######################
model.fit(new_train_x, new_train_y, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2, verbose=1,
          callbacks=[early_stop, save_best_only , reduceLR])


Epoch 1/60


TypeError: in user code:

    /anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /anaconda/envs/py38_tenso

In [101]:
# model = Sequential()
# model.add(LSTM(64, activation='relu', input_shape=(new_train_x.shape[1], new_train_x.shape[2]), return_sequences=True))
# model.add(LSTM(32, activation='relu', return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(new_train_y.shape[1]))
# model.compile(optimizer='adam', loss=SMAPE)

In [133]:
model.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, 32)                4480      
_________________________________________________________________
dense_54 (Dense)             (None, 48)                1584      
_________________________________________________________________
reshape_24 (Reshape)         (None, 24, 2)             0         
Total params: 6,064
Trainable params: 6,064
Non-trainable params: 0
_________________________________________________________________


In [134]:
#######################
prediction=np.zeros((num_power, end_, num_features))
new_test_x=train_x

for i in range(end_//output_window):
    start_=i*output_window
    next_=model.predict(new_test_x[ : , -input_window:, :])
    new_test_x = tf.concat([new_test_x, next_], axis=1)
    print(new_test_x.shape)
    prediction[:, start_: start_ + output_window, :]= next_
    
    
prediction = prediction *size + mini

(60, 1992, 2)
(60, 2016, 2)
(60, 2040, 2)
(60, 2064, 2)
(60, 2088, 2)
(60, 2112, 2)
(60, 2136, 2)


In [240]:
# def SMAPE(true, pred):
#     return np.mean((np.abs(true-pred))*2/(np.abs(true) + np.abs(pred)))*100

# print(SMAPE(tt,prediction))

In [135]:
submission['answer']=prediction.reshape([-1,num_features])
submission

,num_date_time,answer
0,1 2020-08-25 00,862.010952
1,1 2020-08-25 01,854.452139
2,1 2020-08-25 02,840.192468
3,1 2020-08-25 03,814.595619
4,1 2020-08-25 04,814.723061
...,...,...
10075,60 2020-08-31 19,1023.588230
10076,60 2020-08-31 20,1074.868709
10077,60 2020-08-31 21,1174.034525
10078,60 2020-08-31 22,1105.850534


In [136]:
submission.shape

(10080, 2)

In [137]:
submission.to_csv('baseline_submission6.csv', index=False)